In [2]:
import tables
import os
import string
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import json
import tensorflow as tf

In [ ]:
print(len(track_mfcc_list))
track_mfcc_list[:5]

In [ ]:
track_mfcc_list = sorted(track_mfcc_list, key=lambda d: d[1])

In [ ]:
track_mfcc_list[:10]

In [ ]:
mfcc_len_arr = list(map(lambda t: t[1], track_mfcc_list))

In [ ]:
num_bins = 50
mu = 776  # mean of distribution
sigma = 1500
fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(mfcc_len_arr, num_bins, normed=1)

# add a 'best fit' line
y = mlab.normpdf(bins, mu, sigma)
ax.plot(bins, y, '--')
ax.set_xlabel('Smarts')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of IQ: $\mu=100$, $\sigma=15$')

# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.show()


In [ ]:
filtered_track_list = list(filter(lambda d: d[1] >= 300 and d[1] <= 1400, track_mfcc_list))

In [ ]:
num_bins = 50
mu = 776  # mean of distribution
sigma = 1500
fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(tmp, num_bins, normed=1)

# add a 'best fit' line
y = mlab.normpdf(bins, mu, sigma)
ax.plot(bins, y, '--')
ax.set_xlabel('Smarts')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of IQ: $\mu=100$, $\sigma=15$')

# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.show()


In [ ]:
print(filtered_track_list[:10])
print(filtered_track_list[-10:])
final_track_list = list(map(lambda d: d[0], filtered_track_list))

In [ ]:
track_mfcc_dict = {}

def line_splitter(line):
    tmp = line.split(',')
    return (tmp[0], int((len(tmp)-1) / 12), tmp[1:])

def mfcc_reshaper(x_len, org_mfcc):
    
    org_mfcc = list(map(lambda d: float(d), org_mfcc))
    
    mfcc = np.array(org_mfcc).reshape(12, x_len)
    
    if x_len < 700:
        diff = 700 - x_len
        left_pad_len = int(diff / 2)
        right_pad_len = diff - left_pad_len
        mfcc = np.pad(mfcc, ((0,0),(left_pad_len,right_pad_len)), mode='constant', constant_values=0.0)
    else:
        start = int((x_len-700) / 2)
        mfcc = mfcc[:, start:start+700]
        
    return mfcc

for bucket in string.ascii_uppercase:
    
    with open('audio_features/%s.txt' % (bucket), 'r') as fd:
        for track in filter(lambda d:d[1]>=300 and d[1]<=1400, map(line_splitter, map(lambda line: line[:-1], fd.readlines()))):
            track_mfcc_dict[track[0]] = mfcc_reshaper(track[1], track[2])

    print('Done with %s' % bucket)

In [ ]:
len(final_track_list)

In [ ]:
print(len(track_mfcc_dict))
track_mfcc_dict

In [ ]:
for k, v in track_mfcc_dict.items():
    if v.shape[1] != 700:
        print('Error')

print("Done")

In [ ]:
with open('track_tags_labels.json', 'r') as fd:
    track_labels_tags_dict = json.loads(fd.read())

In [ ]:
print(len(track_labels_tags_dict))
print(len(track_mfcc_dict))

In [ ]:
X_all, y_all = np.empty((0, 700 * 12)), np.empty((0, 100))

In [ ]:
for k, v in track_mfcc_dict.items():
    X_all = np.vstack([X_all, v.reshape(-1)])
    y_all = np.vstack([y_all, np.array(track_labels_tags_dict[k]['labels'])])

In [ ]:
print(X_all.shape)
print(y_all.shape)

In [ ]:
def read_target_track_id_list():
    # Read the track ids with top 100 tags
    with open('new_complet_msd_ids.txt', 'r') as fd:
        data = fd.read()
    
    return data.split(',')

def put_target_track_into_buckets(target_track_id_list):
    
    buckets = {}
    
    for upper_letter in string.ascii_uppercase:
         buckets[upper_letter] = list(filter(lambda track_id: track_id[2] == upper_letter, target_track_id_list))
            
    return buckets

target_track_id_list = read_target_track_id_list()
print(len(target_track_id_list))

target_track_id_list = sorted(target_track_id_list)

target_tracks_buckets = put_target_track_into_buckets(target_track_id_list)
print(len(target_tracks_buckets))


In [ ]:
for bucket, tracks in target_tracks_buckets.items():
    X_all, y_all = np.empty((0, 700 * 12)), np.empty((0, 100))
    
    for track in tracks:
        X_all = np.vstack([X_all, track_mfcc_dict[track].reshape(-1)])
        y_all = np.vstack([y_all, np.array(track_labels_tags_dict[track]['labels'])])
        
    print('X_all shape', X_all.shape)
    print('y_all shape', y_all.shape)
    
    np.savetxt("training_data/%s_features.csv" % (bucket), X_all, delimiter=",")
    np.savetxt("training_data/%s_labels.csv" % (bucket), y_all, delimiter=",")

In [3]:
X_all = np.loadtxt('training_data/A_features.csv', delimiter=',')
y_all = np.array(np.loadtxt('training_data/A_labels.csv', delimiter=','), dtype=np.int)

In [4]:
print(X_all.shape)
print(y_all.shape)

(2739, 8400)
(2739, 100)


In [8]:
from sklearn.model_selection import train_test_split
import time

X_not_test, X_test, y_not_rest, y_test = train_test_split(
    X_all, y_all, train_size=.8, random_state=round(time.time()))

X_train, X_val, y_train, y_val = train_test_split(
    X_not_test, y_not_rest, train_size=.85, random_state=round(time.time()))

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(1862, 8400)
(1862, 100)
(329, 8400)
(329, 100)
(548, 8400)
(548, 100)


In [9]:
learning_rate = 0.005
epochs = 40
batch_size = 32

# Network Parameters
n_input = 12 * 700
n_classes = 100
dropout = .8 # Dropout, probability to keep units

# 1. Define Variables and Placeholders
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

def build_model(x, dropout, activation):
    
    x = tf.reshape(x, shape=[-1, 12, 700, 1])
    
    conv1 = tf.layers.conv2d(x, filters=16, kernel_size=(3, 3), activation=activation)
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

    conv2 = tf.layers.conv2d(conv1, 32, kernel_size=(3, 3), activation=activation)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    
    fc1 = tf.contrib.layers.flatten(conv2)

    fc1 = tf.layers.dense(fc1, 128, activation=activation)

    fc2 = tf.layers.dense(fc1, 64, activation=activation)
    
    out = tf.layers.dense(fc2, n_classes)

    return out

predictions = build_model(x, keep_prob, activation=tf.nn.relu)
# 3. Define the loss function
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=predictions, labels=y), name='loss')
    tf.summary.scalar("loss", loss)
    

# # 4. Define the accuracy
# with tf.name_scope("auc_score"):
#     auc_score = tf.metrics.auc(labels=y, predictions=predictions)
#     tf.summary.scalar("auc_score", auc_score)

summary = tf.summary.merge_all()
# 5. Define an optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def feed_next_batch(train_size, batch_size=64):
    
    start = 0
    while start < train_size:
        yield start, start + batch_size
        start += batch_size

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

RUN_NAME = 'audio_nn_1st_run'
writer_train = tf.summary.FileWriter('./log/' + RUN_NAME + '/train', graph=sess.graph)
writer_test = tf.summary.FileWriter('./log/' + RUN_NAME + '/test', graph=sess.graph)

steps = 0
# Keep training until reach max iterations
for epoch in range(epochs):

    indices = np.arange(len(y_train))
    np.random.shuffle(indices)
    X_train, y_train = X_train[indices], y_train[indices]

    for start, end in feed_next_batch(len(X_train), batch_size=batch_size):
        # Run optimization op (backprop)
        batch_x, batch_y = X_train[start:end], y_train[start:end]
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        
        steps += 1
        
        if steps % 20 == 0:

            # Calculate batch loss and accuracy
            train_loss, summary_train = sess.run([loss, summary], 
                                                   feed_dict={x: X_train, y: y_train, keep_prob: 1.0})
            writer_train.add_summary(summary_train, steps)

            print('Step %d' % (steps))
            print ("Training loss:", train_loss, ', training auc: ', 0)

            val_loss, summary_test = sess.run([loss, summary], feed_dict={x: X_val, 
                                                                        y: y_val,
                                                                       keep_prob: 1.0})
            writer_test.add_summary(summary_test, steps)
            print ("Validation loss:", val_loss, ', validation acc: ', 0)
            print('-' * 80)

print ("Optimization Finished!")

# Calculate accuracy for all test samples
# print ("Training AUC:", \
#     sess.run(auc_score, feed_dict={x: X_train,
#                                   y: y_train,
#                                  keep_prob: 1.0}))
# print ("Test AUC:", \
#     sess.run(auc_score, feed_dict={x: X_test,
#                                   y: y_test,
#                                  keep_prob: 1.0}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,8400]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,8400], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/kernelapp.py", lin